In [610]:
import sys
import re
import numbers
import os
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from typing import Tuple, List, Set, Dict, Iterable, Callable, Optional, Union

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline

sns.set_style("darkgrid")

In [611]:
np.random.seed(0)

In [612]:
sys.path.insert(0, "../../")  # Добавляем родительскую директорию в `path`, чтобы python смог найти наши модули

In [613]:
# Проверим, что python теперь их видит:
from modules.preprocessing import MultilabelEncoder, RegexTransformer
from modules.base          import CustomTransformer
from modules.compose       import GroupByTransformer
from modules.outliers      import RangeOutlierDetector, DeltaFromMedianOutlierDetector
from modules.impute        import InterpolationImputer

<div class="alert alert-info">
  <h1><center>Data (2 балла)</center></h1></div>

In [614]:
df = pd.read_csv("data/train.csv", low_memory=False)
df.columns = map(str.lower, df.columns)
df.columns = df.columns.str.replace(r'-', r'_', regex=True)
df.shape

(118936, 33)

In [615]:
pd.set_option("display.max_columns", df.shape[-1])
df.head()

,id,year,loan_limit,gender,approv_in_adv,loan_type,loan_purpose,credit_worthiness,open_credit,business_or_commercial,loan_amount,rate_of_interest,interest_rate_spread,upfront_charges,term,neg_ammortization,interest_only,lump_sum_payment,property_value,construction_type,occupancy_type,secured_by,total_units,income,credit_type,credit_score,co_applicant_credit_type,age,submission_of_application,ltv,region,security_type,status
0,89268,2019,cf,Joint,nopre,type1,p4,l1,nopc,nob/c,216500,3.990,0.8696,1785.38,240.0,not_neg,not_int,not_lpsm,308000.0,sb,pr,home,1U,7380.0,EXP,512,EXP,65-74,to_inst,70.292208,North,direct,0
1,125974,2019,cf,Joint,nopre,type1,p1,l1,nopc,nob/c,606500,3.875,0.2662,3279.06,360.0,not_neg,not_int,not_lpsm,758000.0,sb,pr,home,1U,8580.0,EXP,588,EXP,35-44,not_inst,80.013193,North,direct,0
2,62363,2019,cf,Female,nopre,type1,p1,l1,nopc,nob/c,156500,NaN,NaN,NaN,360.0,neg_amm,not_int,not_lpsm,158000.0,sb,pr,home,1U,4860.0,CRIF,801,EXP,35-44,to_inst,99.050633,North,direct,1
3,106793,2019,cf,Sex Not Available,nopre,type1,p4,l1,nopc,nob/c,306500,NaN,NaN,NaN,360.0,not_neg,not_int,lpsm,NaN,sb,pr,home,1U,6000.0,EQUI,798,EXP,35-44,to_inst,NaN,south,direct,1
4,66272,2019,cf,Joint,nopre,type1,p3,l1,nopc,nob/c,206500,4.375,0.2750,0.00,360.0,not_neg,not_int,not_lpsm,298000.0,sb,sr,home,1U,10500.0,CRIF,554,EXP,>74,not_inst,69.295302,North,direct,0


Базовая информация о датафрейме:

In [616]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118936 entries, 0 to 118935
Data columns (total 33 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         118936 non-null  int64  
 1   year                       118936 non-null  int64  
 2   loan_limit                 116225 non-null  object 
 3   gender                     118936 non-null  object 
 4   approv_in_adv              118209 non-null  object 
 5   loan_type                  118936 non-null  object 
 6   loan_purpose               118836 non-null  object 
 7   credit_worthiness          118936 non-null  object 
 8   open_credit                118936 non-null  object 
 9   business_or_commercial     118936 non-null  object 
 10  loan_amount                118936 non-null  int64  
 11  rate_of_interest           89781 non-null   float64
 12  interest_rate_spread       89625 non-null   float64
 13  upfront_charges            87

Разделим датасет на Х и у:

In [617]:
X = df.drop(columns=["status"])
y = df["status"]

Распределение целевой переменной по классам:

In [618]:
y.value_counts() / len(y)

0    0.753557
1    0.246443
Name: status, dtype: float64

### Train | Test split

Разбейте датасет на тренировочный и тестовый (оставьте 20% для теста, не забудтье зафиксировать random_seed)

In [619]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Разделить конечно разделим... Но для анализа пока будем работать со всеми данными (Х в данном случае). Потом обучим окончательный вариант на тренировочной выборке и применим к обоим частям.

### First look and general info

Отобразите базовую информацию о датасете: размер, тип данных, базовые статистики

Выведем по каждой колонке данных датафрейма статистику (тип данных, процент пропусков, наличие/отсутсвие повторяющихся значений, уникальные категориальные значения).

In [620]:
with open('info_base.txt', 'w') as f:
    for col_name, col_values in X.items():
        f.write(f"< {col_name} >".center(100, "-") + "\n")
        f.write(f"Data type: {col_values.dtype}\n")
        f.write(f"Is unique: {col_values.is_unique}\n")
        f.write(f"% of Nones: {col_values.isna().mean()}\n")

        if col_values.dtype != 'O':
            f.write(f"Statistics:\n{col_values.describe(percentiles=[.01, .05, .25, .5, .75, .95, .99])}\n")
        else:
            f.write(f"Unique values:\n{col_values.value_counts()}\n")

### Data types normalization

Проверьте, все ли признаки имеют правильный тип данных?

Описание данных с сайта:

Функции
- ID = идентификатор клиента заявителя
- year = год подачи заявки
- loan_limit = максимально доступная сумма кредита, которую можно взять
- Gender = тип пола
- approv_in_adv = Кредит предварительно одобрен или нет
- loan_type = Тип кредита
- loan_purpose = причина, по которой вы хотите занять деньги
- Credit_Worthiness = то, как кредитор определяет, что вы не выполните свои долговые обязательства, или насколько вы достойны получить новый кредит.
- open_credit = предварительно одобренный кредит между кредитором и заемщиком. Это позволяет заемщику делать повторные снятия до определенного предела.
- business_or_commercial = Тип использования суммы кредита
- loan_amount = Точная сумма кредита
- rate_of_interest = сумма, которую кредитор взимает с заемщика, и процент от основной суммы долга.
- Interest_rate_spread = разница между процентной ставкой, которую финансовое учреждение платит вкладчикам, и процентной ставкой, которую оно получает по кредитам.
- Upfront_charges = Комиссия, уплачиваемая кредитору заемщиком в качестве вознаграждения за выдачу нового кредита
- term = период погашения кредита
- Neg_ammortization = относится к ситуации, когда заемщик кредита вносит платеж меньше стандартного платежа, установленного банком.
- interest_only = сумма процентов
- lump_sum_payment = сумма денег, которая выплачивается одним платежом, а не в рассрочку.
- property_value = текущая стоимость будущих выгод, связанных с владением недвижимостью
- Construction_type = Тип вспомогательной конструкции
- occupancy_type = классификации относятся к категоризации структур на основе их использования
- Secured_by = Тип залога
- total_units = количество единиц
- income = относится к сумме денег, имущества и других денежных переводов, полученных в течение установленного периода времени.
- credit_type = тип кредита
- co-applicant_credit_type = дополнительное лицо, участвующее в процессе подачи заявки на кредит. И заявитель, и созаявитель подают и подписывают заявку на получение кредита.
- age = возраст заявителя
- submit_of_application = Убедитесь, что заявка заполнена или нет
- LTV = жизненная ценность (LTV) — это прогноз чистой прибыли.
- Region = место заявителя
- Security_Type = Тип залога
- status = статус займа (одобрен/отклонен)

Неверный тип данных имеет колонка credit_score. Поработаем с ней.

In [621]:
unique_values = df['credit_score'].unique()
numeric_values = pd.to_numeric(unique_values, errors='coerce')
numeric_values = numeric_values[~pd.isna(numeric_values)]
numeric_values.min(), numeric_values.max()

(500.0, 900.0)

При этом нечисловые значения имеют вид:

In [622]:
non_numeric_values = list(filter(lambda x: not (x.isnumeric()), unique_values))
non_numeric_values

['6_32',
 '_576',
 '_566',
 '_764',
 '640_',
 '71_2',
 '_801',
 '626_',
 '75_8',
 '_675']

Сделаем вывод, что знак нижнего подчёркивания можно просто убрать.

In [623]:
def clean_and_convert(value):
    clean_value = re.sub(r'[^\d-]+', '', value)
    return int(clean_value)

In [624]:
X['credit_score'] = X['credit_score'].apply(clean_and_convert)

Колонки id, year просто дропнем. Они не нужны.

In [625]:
X.drop(['id', 'year'], axis=1, inplace=True)

### Extreme values detection

Проверьте, есть ли аномальные данные? Постройте базовые визуализации там, где это уместно

In [626]:
def describe_numeric(X: pd.DataFrame, features: Union[str, Iterable[str], None] = None):
    
    if isinstance(features, str):
        X = X.loc[:, [features]]
    elif isinstance(features, Iterable):
        X = X.loc[:, features]

    for col_name, col_values in X.items():
        print(f"< {col_name} >".center(117, "-"))

        _, (ax0, ax1, ax2, ax3) = plt.subplots(nrows=4, figsize=(11.5, 12))

        _ = ax0.boxplot(col_values.dropna(), labels=[""], vert=False, widths=0.8, showmeans=True)
        _ = ax1.boxplot(col_values.dropna(), labels=[""], vert=False, widths=0.8, showmeans=True)
        _ = ax2.hist(col_values.dropna())
        _ = ax3.hist(col_values.dropna())

        ax0.set(title=f"boxplot of {col_name}")
        ax1.set(title=f"boxplot of {col_name}, log axes", xscale="log")
        ax2.set(title=f"histogram of {col_name}")
        ax3.set(title=f"histogram of {col_name}, log axes", yscale="log", xscale="log")

        plt.show()

In [627]:
# describe_numeric(X=X.select_dtypes(np.number))

В следующих столбцах имеются выбросы (крайнее значение 'сильно' превышает последний перцентиль):
- loan_amount
- interest_rate_spread
- upfront_charges
- term
- property_value
- income
- ltv

### Missing values detection

Есть ли пропущенные данные?

В следующих столбцах имеются отсутствующие значения:
- loan_limit
- approv_in_adv
- loan_purpose
- rate_of_interest
- interest_rate_spread
- upfront_charges
- term
- neg_ammortization
- property_value
- income
- age
- submission_of_application
- ltv

Отметим, что дополнительные пропуки могут появиться в колонке loan_amount после удаления экстримальных значений

Сразу удалим 0 в колонке income и обозначим его пропуском.

In [628]:
X.loc[X['income'].eq(0), ['income']] = np.nan

---

<div class="alert alert-info">
  <h1><center>Preprocessing (3 балла)</center></h1></div>

# <center>Numerical features</center>

### Deal with extreme values

Напишите / используйте функции / трансформеры, которые будут "исправлять" экстримальные значения (какой способ лучше подходит? почему?)

Удалять экстримальные значения будем с помощью межквартильного расстояния. С другими не сильно и знаком...

In [629]:
for col_name, _ in X.select_dtypes(np.number).items():
    Q1 = X[col_name].dropna().quantile(.25)
    Q3 = X[col_name].dropna().quantile(.75)
    IQR = Q3 - Q1

    X[col_name] = X[col_name].where(((X[col_name] >= Q1 - 1.5 * IQR)&(X[col_name] <= Q3 + 1.5 * IQR)))

### Deal with missing values

Напишите/используйте функции/трансформеры для заполнения пропущенных значений. Какова логика? Напишите краткое пояснение, почему Вы выбрали тот или иной подход

Для числовых признаков применим один из двух трансформеров. На медианное значение будем менять те фичи, где пропусков мало. На "соседнее" значение будем менять те фичи, где пропусков много. Кстати о пропусках... Инфа о них лежит в файле info_after_drop.txt.

In [630]:
with open('info_after_drop.txt', 'w') as f:
    for col_name, col_values in X.select_dtypes(np.number).items():
        f.write(f"< {col_name} >".center(100, "-") + "\n")
        f.write(f"Data type: {col_values.dtype}\n")
        f.write(f"Is unique: {col_values.is_unique}\n")
        f.write(f"% of Nones: {col_values.isna().mean()}\n")

        if col_values.dtype != 'O':
            f.write(f"Statistics:\n{col_values.describe(percentiles=[.01, .05, .25, .5, .75, .95, .99])}\n")
        else:
            f.write(f"Unique values:\n{col_values.value_counts()}\n")

Для колонок с менее чем 20 % пропусков применим заполение средним. В другом случае попробуем сделать похожим на соседей. Также выбросим колонку со сроком кредитования. Почти все и так берут на год...

In [631]:
X.drop('term', axis=1, inplace=True)

In [632]:
median_imputer = SimpleImputer(strategy='median')
knn_imputer = KNNImputer(n_neighbors=5)

In [633]:
X[['loan_amount',
   'property_value',
   'income',
   'ltv']] = median_imputer.fit_transform(X[['loan_amount',
                                             'property_value',
                                             'income',
                                             'ltv']])
X[['rate_of_interest',
   'interest_rate_spread',
   'upfront_charges']] = knn_imputer.fit_transform(X[['rate_of_interest',
                                                      'interest_rate_spread',
                                                      'upfront_charges']])

Для оставшихся категориальных фич сделаем заполнение с помощью самого частого знечения.

In [634]:
freq_imputer = SimpleImputer(strategy='most_frequent')

In [635]:
X[['loan_limit',
   'approv_in_adv',
   'loan_purpose',
   'neg_ammortization',
   'age',
   'submission_of_application']] = freq_imputer.fit_transform(X[['loan_limit',
                                                                 'approv_in_adv',
                                                                 'loan_purpose',
                                                                 'neg_ammortization',
                                                                 'age',
                                                                 'submission_of_application']])

Итоговый обзор после удаления пропусковых и заполнения отсутсвующих значений:

In [636]:
X.head(2)

,loan_limit,gender,approv_in_adv,loan_type,loan_purpose,credit_worthiness,open_credit,business_or_commercial,loan_amount,rate_of_interest,interest_rate_spread,upfront_charges,neg_ammortization,interest_only,lump_sum_payment,property_value,construction_type,occupancy_type,secured_by,total_units,income,credit_type,credit_score,co_applicant_credit_type,age,submission_of_application,ltv,region,security_type
0,cf,Joint,nopre,type1,p4,l1,nopc,nob/c,216500.0,3.990,0.8696,1785.38,not_neg,not_int,not_lpsm,308000.0,sb,pr,home,1U,7380.0,EXP,512,EXP,65-74,to_inst,70.292208,North,direct
1,cf,Joint,nopre,type1,p1,l1,nopc,nob/c,606500.0,3.875,0.2662,3279.06,not_neg,not_int,not_lpsm,758000.0,sb,pr,home,1U,8580.0,EXP,588,EXP,35-44,not_inst,80.013193,North,direct


In [637]:
with open('info_аfter_fill_na.txt', 'w') as f:
    for col_name, col_values in X.items():
        f.write(f"< {col_name} >".center(100, "-") + "\n")
        f.write(f"Data type: {col_values.dtype}\n")
        f.write(f"Is unique: {col_values.is_unique}\n")
        f.write(f"% of Nones: {col_values.isna().mean()}\n")

        if col_values.dtype != 'O':
            f.write(f"Statistics:\n{col_values.describe(percentiles=[.01, .05, .25, .5, .75, .95, .99])}\n")
        else:
            f.write(f"Unique values:\n{col_values.value_counts()}\n")

---

# <center>Categorical Features</center>

Какие признаки и как Вы будете кодировать?

### Encode nominal featuers

К номинальным фичам я отнесу:
- loan_limit
- gender
- approv_in_adv
- loan_type
- loan_purpose
- credit_worthiness
- open_credit
- business_or_commercial
- neg_ammortization
- interest_only
- lump_sum_payment
- construction_type
- occupancy_type
- secured_by
- credit_type
- co_applicant_credit_type
- submission_of_application
- region
- security_type

Номинальные фичи можно кодировать OneHotEncoder

In [638]:
one_hot = OneHotEncoder(sparse_output=False, handle_unknown="ignore").set_output(transform="pandas")

In [639]:
encode_df = one_hot.fit_transform(X[['loan_limit',
                                     'gender',
                                     'approv_in_adv',
                                     'loan_type',
                                     'loan_purpose',
                                     'credit_worthiness',
                                     'open_credit',
                                     'business_or_commercial',
                                     'neg_ammortization',
                                     'interest_only',
                                     'lump_sum_payment',
                                     'construction_type',
                                     'occupancy_type',
                                     'secured_by',
                                     'credit_type',
                                     'co_applicant_credit_type',
                                     'submission_of_application',
                                     'region',
                                     'security_type']])

In [640]:
X = X.join(encode_df)
X.drop(['loan_limit',
        'gender',
        'approv_in_adv',
        'loan_type',
        'loan_purpose',
        'credit_worthiness',
        'open_credit',
        'business_or_commercial',
        'neg_ammortization',
        'interest_only',
        'lump_sum_payment',
        'construction_type',
        'occupancy_type',
        'secured_by',
        'credit_type',
        'co_applicant_credit_type',
        'submission_of_application',
        'region',
        'security_type'],
        axis=1,
        inplace=True)

### Encode ordinal features

К порядковым фичам я отнесу:
- total_units
- age

In [641]:
total_units_enc = OrdinalEncoder(categories=[['1U', '2U', '3U', '4U'],
                                             ['<25', '25-34', '35-44', '45-54', '55-64', '65-74', '>74']],
                                 handle_unknown='use_encoded_value',
                                 unknown_value=-1)

In [642]:
X[['total_units', 'age']] = total_units_enc.fit_transform(X[['total_units', 'age']])

In [643]:
X.shape

(118936, 58)

In [644]:
X.head(2)

,loan_amount,rate_of_interest,interest_rate_spread,upfront_charges,property_value,total_units,income,credit_score,age,ltv,loan_limit_cf,loan_limit_ncf,gender_Female,gender_Joint,gender_Male,gender_Sex Not Available,...,secured_by_home,secured_by_land,credit_type_CIB,credit_type_CRIF,credit_type_EQUI,credit_type_EXP,co_applicant_credit_type_CIB,co_applicant_credit_type_EXP,submission_of_application_not_inst,submission_of_application_to_inst,region_North,region_North-East,region_central,region_south,security_type_Indriect,security_type_direct
0,216500.0,3.990,0.8696,1785.38,308000.0,0.0,7380.0,512,5.0,70.292208,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
1,606500.0,3.875,0.2662,3279.06,758000.0,0.0,8580.0,588,2.0,80.013193,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


Сделаем последний обзор (без пайплайна). Смотреть info_final_without_pipe.txt

In [645]:
with open('info_final_without_pipe.txt', 'w') as f:
    for col_name, col_values in X.items():
        f.write(f"< {col_name} >".center(100, "-") + "\n")
        f.write(f"Data type: {col_values.dtype}\n")
        f.write(f"Is unique: {col_values.is_unique}\n")
        f.write(f"% of Nones: {col_values.isna().mean()}\n")

        if col_values.dtype != 'O':
            f.write(f"Statistics:\n{col_values.describe(percentiles=[.01, .05, .25, .5, .75, .95, .99])}\n")
        else:
            f.write(f"Unique values:\n{col_values.value_counts()}\n")

In [646]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118936 entries, 0 to 118935
Data columns (total 58 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   loan_amount                         118936 non-null  float64
 1   rate_of_interest                    118936 non-null  float64
 2   interest_rate_spread                118936 non-null  float64
 3   upfront_charges                     118936 non-null  float64
 4   property_value                      118936 non-null  float64
 5   total_units                         118936 non-null  float64
 6   income                              118936 non-null  float64
 7   credit_score                        118936 non-null  int64  
 8   age                                 118936 non-null  float64
 9   ltv                                 118936 non-null  float64
 10  loan_limit_cf                       118936 non-null  float64
 11  loan_limit_ncf            

---

<div class="alert alert-info">
  <h1><center>Pipeline (3 балла)</center></h1></div>

Соберите pipeline, который будет реализовывать все предыдущие трансформации

---

<div class="alert alert-info">
  <h1><center>Model (2 балла)</center></h1></div>

Выберите любую из пройденных моделей и примените её к обработанному датасету

Оцените модель при помощи изученных метрик

Что можно сделать, чтобы оптимизировать полученную модель?

---

# <center>Удачи!</center>